In [1]:
# Adapted from : https://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html

import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import time
import os
import urllib.request
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
def ptb_iterator(raw_data, batch_size, num_steps, steps_ahead=1):

    raw_data = np.array(raw_data, dtype=np.int32)
    data_len = len(raw_data)
    batch_len = data_len // batch_size
    data = np.zeros([batch_size, batch_len], dtype=np.int32)
    offset = 0
    if data_len % batch_size:
        offset = np.random.randint(0, data_len % batch_size)
    for i in range(batch_size):
        data[i] = raw_data[batch_len * i + offset:batch_len * (i + 1) + offset]
    epoch_size = (batch_len - steps_ahead) // num_steps
    if epoch_size == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")
    for i in range(epoch_size):
        x = data[:, i*num_steps:(i+1)*num_steps]
        y = data[:, i*num_steps+1:(i+1)*num_steps+steps_ahead]
        yield (x, y)
    if epoch_size * num_steps < batch_len - steps_ahead:
        yield (data[:, epoch_size*num_steps : batch_len - steps_ahead], data[:, epoch_size*num_steps + 1:])


def shuffled_ptb_iterator(raw_data, batch_size, num_steps):
    raw_data = np.array(raw_data, dtype=np.int32)
    r = len(raw_data) % num_steps
    if r:
        n = np.random.randint(0, r)
        raw_data = raw_data[n:n + len(raw_data) - r]
    raw_data = np.reshape(raw_data, [-1, num_steps])
    np.random.shuffle(raw_data)
    num_batches = int(np.ceil(len(raw_data) / batch_size))
    for i in range(num_batches):
        data = raw_data[i*batch_size:min(len(raw_data), (i+1)*batch_size),:]
        yield (data[:,:-1], data[:,1:])

In [3]:
"""
Load and process data, utility functions
"""

file_url = 'https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt'
file_name = 'tinyshakespeare.txt'
if not os.path.exists(file_name):
    urllib.request.urlretrieve(file_url, file_name)

with open(file_name,'r') as f:
    raw_data = f.read()
    print("Data length:", len(raw_data))

vocab = set(raw_data)
vocab_size = len(vocab)
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))

data = [vocab_to_idx[c] for c in raw_data]
del raw_data

def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield ptb_iterator(data, batch_size, num_steps)

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def train_network(g, num_epochs, num_steps = 200, batch_size = 32, verbose = True, save=False):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            training_loss = 0
            steps = 0
            training_state = None
            for X, Y in epoch:
                steps += 1

                if X.shape[1] != g['x'].shape[1]:
                    break 
                    
                feed_dict={g['x']: X, g['y']: Y}
                if training_state is not None:
                    feed_dict[g['init_state']] = training_state
                training_loss_, training_state, _ = sess.run([g['total_loss'],
                                                      g['final_state'],
                                                      g['train_step']],
                                                             feed_dict)
                training_loss += training_loss_
            if verbose:
                print("Average training loss for Epoch", idx, ":", training_loss/steps)
            training_losses.append(training_loss/steps)

        if isinstance(save, str):
            g['saver'].save(sess, save)

    return training_losses

Data length: 1115394


In [4]:
def build_basic_rnn_graph_with_list(
    state_size = 100,
    num_classes = vocab_size,
    batch_size = 32,
    num_steps = 200,
    learning_rate = 1e-4):

    reset_graph()

    x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
    y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')

    x_one_hot = tf.one_hot(x, num_classes) # <n, t, f>
    rnn_inputs = [tf.squeeze(i,squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, axis=1)]

    cell = tf.contrib.rnn.LSTMCell(state_size)
    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=init_state)

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]

    y_as_list = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(y, num_steps, axis=1)] # t arrays of tensor <n, o>

    loss_weights = [tf.ones([batch_size]) for i in range(num_steps)]
    losses = tf.contrib.legacy_seq2seq.sequence_loss_by_example(logits, y_as_list, loss_weights)
    total_loss = tf.reduce_mean(losses)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

    return dict(
        x = x,
        y = y,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step
    )

In [5]:
t = time.time()
g = build_basic_rnn_graph_with_list()
print("It took", time.time() - t, "seconds to build the graph.")

It took 23.717364072799683 seconds to build the graph.


In [7]:
t = time.time()
train_network(g, 30)
print("It took", time.time() - t, "seconds to train for 30 epochs.")

Average training loss for Epoch 0 : 3.65346962384
Average training loss for Epoch 1 : 3.28766210692
Average training loss for Epoch 2 : 3.26015863419
Average training loss for Epoch 3 : 3.21657994815
Average training loss for Epoch 4 : 3.16339379719
Average training loss for Epoch 5 : 3.11113892828
Average training loss for Epoch 6 : 3.05933697156
Average training loss for Epoch 7 : 3.00586072649
Average training loss for Epoch 8 : 2.94736801829
Average training loss for Epoch 9 : 2.88377714702
Average training loss for Epoch 10 : 2.81776434898
Average training loss for Epoch 11 : 2.75292940685
Average training loss for Epoch 12 : 2.6927074119
Average training loss for Epoch 13 : 2.63938294138
Average training loss for Epoch 14 : 2.5932237666
Average training loss for Epoch 15 : 2.55341040475
Average training loss for Epoch 16 : 2.51890320914
Average training loss for Epoch 17 : 2.4888315664
Average training loss for Epoch 18 : 2.46234257698
Average training loss for Epoch 19 : 2.43876